# Unit 4: Orthogonality, Projections, and Least Squares

In this unit:
1. Orthogonal and orthonormal vectors
1. Orthogonal vector spaces and relations for the fundamental subspaces
1. Grahm-Schmidt
1. QR factorization.
1. Projections
1. Least Squares

Two vectors $u$ and $v$ are said to be **orthogonal** if $u \cdot v = 0$.

In [1]:
using LinearAlgebra
u = [1,1];
v =[-1,1];
u⋅v

0

A collection of vectors $\{v_1,\ldots,v_k\}$ is said to be orthogonal if every pair $v_i$ and $v_j$ for $i \neq j$ is orthogonal.

Further the collection is called **orthonormal** if $||v_i|| = 1$ for each $i$.

Hence for an orthonormal collection of vectors,
$$
v_i'v_j =
\begin{cases}
1 & i=j \\
0 & i \neq j \\
\end{cases}
$$


Example: The standard $n$-vectors $e_1,\ldots,e_n$ are orthonormal. But we can also rotate them or flip them and still keep orthonormal vectors.

**Claim:** Orthonormal vectors are linearly indepednent.

## Orthogonal Vector Spaces

Vector spaces ${\cal V}$ and ${\cal W}$ are orthogonal if $v^Tw = 0$ for every $v \in {\cal V}$ and $w \in {\cal W}$.

## Projecting onto a line

Projecting the vector $u$ onto the vector $v$ we get
$$
p = \hat{x} \, v = \frac{v^T u}{v^T v} \, v.
$$

## Gram - Schmidt

(Alg 5.1, pp 97 [VMLS]:

Given linearly $n$-vectors $v_1,\ldots,v_k$, for $i=1,\ldots,k$:

1. Orthogonalization: Set
$$
\tilde{q}_i = v_i - (q_1^T v_i) q_1 - \ldots - (q_{i-1}^T v_i) q_{i-1}
$$

1. If $\tilde{q}_i = 0$ quit saying "linearly dependent".
1. Normalization: Set $q_i = \tilde{q}_i/||\tilde{q}_i||$.

**todo**: Implement Gram-Schmidt

A matrix with orthonormal columns $A$ satisfies,

$$
A^TA = I
$$

This of $A$ as $n \times k$ so the $I$ is $k \times k$.

If $A$ is in addition square then it is called an orthogonal matrix and $A^T = A^{-1}$.

1. $|| A x|| = ||x||$
1. $(Ax)^T(Ay) = x^Ty$
1. $\angle (Ax, Ay) = \angle (x,y)$

## QR Factorization

In [41]:
using LinearAlgebra
A =[1 0 1;
    2 0 1;
    1 1 1;
    2 3 4]
F = qr(A)

LinearAlgebra.QRCompactWY{Float64,Array{Float64,2}}
Q factor:
4×4 LinearAlgebra.QRCompactWYQ{Float64,Array{Float64,2}}:
 -0.316228  -0.309965   0.642353  -0.625543
 -0.632456  -0.61993   -0.204385   0.417029
 -0.316228   0.132842  -0.700749  -0.625543
 -0.632456   0.708492   0.233583   0.208514
R factor:
3×3 Array{Float64,2}:
 -3.16228  -2.21359  -3.79473
  0.0       2.25832   2.03691
  0.0       0.0       0.671551

In [34]:
Q = F.Q;
R = F.R;

In [35]:
Q

4×4 LinearAlgebra.QRCompactWYQ{Float64,Array{Float64,2}}:
 -0.316228  -0.309965   0.642353  -0.625543
 -0.632456  -0.61993   -0.204385   0.417029
 -0.316228   0.132842  -0.700749  -0.625543
 -0.632456   0.708492   0.233583   0.208514

In [33]:
R

3×3 Array{Float64,2}:
 -3.16228  -2.21359  -3.79473
  0.0       2.25832   2.03691
  0.0       0.0       0.671551

In [38]:
rank(Q[:,1:3])

3

In [36]:
F.Q'*F.Q

4×4 Array{Float64,2}:
  1.0          -5.55112e-17  8.32667e-17  -1.11022e-16
 -5.55112e-17   1.0          5.55112e-17  -1.66533e-16
  8.32667e-17   5.55112e-17  1.0           3.46945e-17
 -1.11022e-16  -1.66533e-16  3.46945e-17   1.0